<a href="https://colab.research.google.com/github/emyesme/CalcificationDetection/blob/feature-pm/proi_file_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# https://medium.com/@robertbracco1/configuring-google-colab-like-a-pro-d61c253f7573#a642
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [2]:
import os 

#first put a shortcut in your drive to the image processing folder


DATA_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Image Processing and Analysis 2022',
                        'projects',
                        'Calcification Detection',
                        'dataset')

RESULTS_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Results', 
                        'proi_files', 
                        'train-validation-test')


print(os.listdir(RESULTS_DIR))

results_file = os.listdir(RESULTS_DIR)


['MC.w12.validation.proi', 'MC.w12.train.proi', 'noMC.w12.validation.proi', 'noMC.w12.train.proi', 'MC.w12.test.proi', 'noMC.w12.test.proi', 'MC.w24.validation.proi', 'noMC.w24.validation.proi', 'MC.w24.train.proi', 'noMC.w24.train.proi', 'MC.w24.test.proi', 'noMC.w24.test.proi', 'noMC.w48.validation.proi', 'MC.w48.validation.proi', 'MC.w48.test.proi', 'MC.w48.train.proi', 'noMC.w48.test.proi', 'noMC.w48.train.proi', 'MC.w96.train.proi', 'noMC.w96.test.proi', 'MC.w96.validation.proi', 'noMC.w96.validation.proi', 'noMC.w96.train.proi', 'MC.w96.test.proi']


In [3]:
import numpy as np
import pandas as pd
import os
import cv2
#!pip install fastprogress
from fastprogress import master_bar, progress_bar
import gc
import time

In [5]:
from google.colab.patches import cv2_imshow
#import plt for display
import matplotlib.pyplot as plt

#go into de directory of the images

# this have 3 outputs root directory, the folders in the path and the files in the path.
# we ignore _ the two first because we are not interested in those
_, _, images = next(os.walk(os.path.join(DATA_DIR,'images')))
_, _, breastMasks = next(os.walk(os.path.join(DATA_DIR,'masks')))
_, _, groundTruths = next(os.walk(os.path.join(DATA_DIR, 'groundtruths')))

images.sort()
breastMasks.sort()
groundTruths.sort()

# read numbers of normal images
normals = []
with open(os.path.join(DATA_DIR,'normals.txt')) as f:
    for line in f:
        normals.append(line[:-1])

In [6]:
def sliding_window(image, stepSize, windowSize, groundTruth, mask):
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            # This yields the current window, label: 1 if MC and 0 if not MC, and Background: 1 if background, 0 if not background
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]], str(int(np.sum(groundTruth[y:y + windowSize[1], x:x + windowSize[0]])>0)), str(int(np.sum(mask[y:y + windowSize[1], x:x + windowSize[0]])==0)))

In [7]:
import copy
import pandas as pd

groundTruthsDataFrame = pd.read_csv(os.path.join('/content',
                              'drive',
                              'MyDrive',
                              'Results',
                              'groundTruthStats.csv'))

groundTruthsDataFrame['key']= [i.split('_')[0] for i in groundTruthsDataFrame['name'].values.tolist()]

row_centroids = []
for index, row in groundTruthsDataFrame.iterrows():
  centroids = []
  for key, value in eval(row['centroids']).items():
    if key == 0:
      continue
    x = round(value['x'])
    y = round(value['y'])
    centroid = (x, y)
    centroids.append(centroid)
  row_centroids.append(centroids)

groundTruthsDataFrame['tuple_centroids'] = row_centroids


In [4]:
df_train_test = pd.read_csv(os.path.join('/content',
                              'drive',
                              'MyDrive',
                              'Results',
                              'standard_partitions.csv'), index_col=0)
train_keys = df_train_test.loc[df_train_test.partition == 'train'].sample(frac=0.8, random_state=1)['image_id'].values
validation_keys= [i for i in df_train_test.loc[df_train_test.partition == 'train']['image_id'].values if i not in train_keys]
test_keys = df_train_test.loc[df_train_test.partition == 'test']['image_id'].values

print(len(train_keys))
print(len(validation_keys))
print(len(test_keys))


168
42
200


In [5]:
train_keys

array([22670278, 53582422, 22679036, 53586869, 22613848, 24065887,
       22613796, 51048891, 53586896, 51048918, 27829134, 20587200,
       22670347, 20587758, 24055573, 50997488, 51049134, 24055877,
       22613970, 22613822, 24055382, 22580732, 50993616, 53580979,
       22614074, 50997080, 20587492, 50996945, 22580680, 24065289,
       30011850, 30011553, 50994814, 22580654, 20587664, 30011484,
       50998349, 24065860, 22580098, 50997434, 24055654, 53587508,
       30318067, 50997107, 26933859, 24065707, 53582656, 53581433,
       50993697, 53586724, 50994706, 53582476, 50994616, 50993814,
       24055931, 20587612, 51048765, 53581237, 20587784, 53582683,
       24065914, 22613996, 50993922, 53582791, 24055328, 53581060,
       24055274, 53587481, 24055176, 27829215, 50996999, 24065461,
       50993949, 22614127, 20587690, 20587226, 50997461, 51049053,
       22678953, 51048945, 24055806, 50996709, 24055203, 27829188,
       50994535, 53582449, 26933830, 51049249, 50994895, 20587

In [14]:
(validation_keys != test_keys)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


True

In [ ]:
print(image.split('_')[0])
if int('20587054') in list(train_keys):
  print('si')
train_keys

NameError: ignored

In [15]:

windowsWH = [12, 24, 48, 96]

for w in windowsWH:
  
    winW=w
    winH=w


    """    roi_width = xx; roi_height = xx; scale_factor = xx
        <image_filename> <px> <py>"""

    df_1 = pd.DataFrame(columns = [f'roi_width = {winW}', f' roi_height = {winH}', ' scale_factor = 1'])
    df_1.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.train.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.validation.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.test.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.train.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.validation.proi'), index=False, sep=';')
    df_1.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.test.proi'), index=False, sep=';')

    df_2 = pd.DataFrame(columns = ['<image_filename>', '<px>', '<py>'])
    df_2.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.train.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.validation.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'noMC.w{winW}.test.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.train.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.validation.proi'), index=False, mode='a', sep=' ')
    df_2.to_csv(os.path.join(RESULTS_DIR, f'MC.w{winW}.test.proi'), index=False, mode='a', sep=' ')

    already = []

    for image, breastMask, groundTruth in zip(progress_bar(images), breastMasks, groundTruths):
        start = time.time()

        mc_list = []
        no_mc_list = []


        count_neg = 0
        count_pos = 0
        count_bkg = 0


        # 20588020, 7717, 5328, 3787, 5725, 3859, 6934, 50995872
        # digits = '20587372'

        # if ((digits in image) and (digits in breastMask) and ('mask' in breastMask)):
        if image.split('_')[0] not in already:

            #if image not in already:
            print("image: ", image)
            #upload images
            img = cv2.imread(os.path.join(DATA_DIR,'images',image))
            imgMask = cv2.imread(os.path.join(DATA_DIR, 'masks', breastMask), cv2.IMREAD_GRAYSCALE)
            imgGroundTruth = cv2.imread(os.path.join(DATA_DIR, 'groundtruths', groundTruth), cv2.IMREAD_GRAYSCALE)


            gtRow = groundTruthsDataFrame.loc[groundTruthsDataFrame['key'] == image.split('_')[0]]

            if int(image.split('_')[0]) in list(train_keys):
              filename_MC = f'MC.w{winW}.train.proi'
              filename_noMC = f'noMC.w{winW}.train.proi'
            elif int(image.split('_')[0]) in list(test_keys):
              filename_MC = f'MC.w{winW}.test.proi'
              filename_noMC = f'noMC.w{winW}.test.proi'
            else:
              filename_MC = f'MC.w{winW}.validation.proi'
              filename_noMC = f'noMC.w{winW}.validation.proi'

            for centroid in gtRow['tuple_centroids'].values[0]:
                if ((centroid[1]+(winW/2)) < img.shape[0]) and ((centroid[0]+(winW/2)) < img.shape[1]) and ((centroid[1]-(winW/2)) > 0) and ((centroid[0]-(winW/2)) > 0):
                    mc_list.append([image, int(centroid[0]-winW/2), int(centroid[1]-winW/2)])
                    count_pos += 1 
                
            df_mc = pd.DataFrame(mc_list,columns=['<image_filename>', '<px>', '<py>'])
            df_mc.to_csv(os.path.join(RESULTS_DIR, filename_MC), index=False, header=False, mode='a', sep=' ')
            
            print('File MC done')

            del df_mc
            
            # Label is 1 if MC and 0 in no MC. Background is  1 if background and 0 if not
            for (x, y, window, label, background) in sliding_window(img, stepSize=int(winW/3)*2, windowSize=(winW, winH), groundTruth=imgGroundTruth, mask=imgMask):
                if window.shape[0] != winH or window.shape[1] != winW:
                    continue

                if bool(int(background)):
                    count_bkg += 1
                    continue

                if label == '0':
                  no_mc_list.append([image, x, y])
                  count_neg += 1

            df_no_mc = pd.DataFrame(no_mc_list,columns=['<image_filename>', '<px>', '<py>'])
            df_no_mc.to_csv(os.path.join(RESULTS_DIR, filename_noMC), index=False, header=False,  mode='a', sep=' ')

            print('File noMC done')

            print('Number of backgrounds ignored: ', count_bkg)
            print('Number of positives: ', count_pos)
            print('Number of negatives: ', count_neg)




            del df_no_mc, img, imgMask, imgGroundTruth, window
            gc.collect()

            print("Time taken for the image: ", time.time() - start)

    # machine learning must be applied for the classification of the features extracted



image:  20586908_6c613a14b80a8591_MG_R_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  130719
Number of positives:  39
Number of negatives:  80688
Time taken for the image:  7.614384651184082
image:  20586934_6c613a14b80a8591_MG_L_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  142536
Number of positives:  58
Number of negatives:  68803
Time taken for the image:  7.531024932861328
image:  20586960_6c613a14b80a8591_MG_R_ML_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  120984
Number of positives:  43
Number of negatives:  90412
Time taken for the image:  8.086669683456421
image:  20586986_6c613a14b80a8591_MG_L_ML_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  133506
Number of positives:  14
Number of negatives:  78055
Time taken for the image:  7.16742730140686
image:  20587054_b6a4f750c6df4f90_MG_R_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  90083
Number of positives:

image:  53580885_51bec6477a7898b9_MG_R_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  157449
Number of positives:  14
Number of negatives:  54119
Time taken for the image:  7.584939479827881
image:  53580979_4c341dad22471922_MG_L_ML_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  89472
Number of positives:  0
Number of negatives:  42913
Time taken for the image:  5.506087779998779
image:  53581006_4c341dad22471922_MG_R_ML_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  91316
Number of positives:  0
Number of negatives:  41069
Time taken for the image:  4.940829277038574
image:  53581033_4c341dad22471922_MG_L_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  95616
Number of positives:  0
Number of negatives:  36769
Time taken for the image:  7.328706502914429
image:  53581060_4c341dad22471922_MG_R_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  97377
Number of positives:  0
N

image:  20586908_6c613a14b80a8591_MG_R_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  32253
Number of positives:  39
Number of negatives:  20173
Time taken for the image:  1.7444798946380615
image:  20586934_6c613a14b80a8591_MG_L_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  34966
Number of positives:  58
Number of negatives:  17408
Time taken for the image:  1.7201809883117676
image:  20586960_6c613a14b80a8591_MG_R_ML_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  29831
Number of positives:  43
Number of negatives:  22579
Time taken for the image:  1.7947595119476318
image:  20586986_6c613a14b80a8591_MG_L_ML_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  32923
Number of positives:  14
Number of negatives:  19603
Time taken for the image:  1.7952969074249268
image:  20587054_b6a4f750c6df4f90_MG_R_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  22268
Number of positives

image:  20586908_6c613a14b80a8591_MG_R_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  7951
Number of positives:  39
Number of negatives:  5015
Time taken for the image:  1.3186321258544922
image:  20586934_6c613a14b80a8591_MG_L_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  8609
Number of positives:  58
Number of negatives:  4320
Time taken for the image:  0.9035348892211914
image:  20586960_6c613a14b80a8591_MG_R_ML_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  7353
Number of positives:  43
Number of negatives:  5591
Time taken for the image:  0.933356761932373
image:  20586986_6c613a14b80a8591_MG_L_ML_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  8109
Number of positives:  14
Number of negatives:  4927
Time taken for the image:  0.8943612575531006
image:  20587054_b6a4f750c6df4f90_MG_R_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  5448
Number of positives:  0
Numbe

image:  20586908_6c613a14b80a8591_MG_R_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  1907
Number of positives:  39
Number of negatives:  1217
Time taken for the image:  0.6593284606933594
image:  20586934_6c613a14b80a8591_MG_L_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  2062
Number of positives:  58
Number of negatives:  1033
Time taken for the image:  0.6586275100708008
image:  20586960_6c613a14b80a8591_MG_R_ML_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  1760
Number of positives:  43
Number of negatives:  1349
Time taken for the image:  0.6588325500488281
image:  20586986_6c613a14b80a8591_MG_L_ML_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  1938
Number of positives:  14
Number of negatives:  1227
Time taken for the image:  0.6144185066223145
image:  20587054_b6a4f750c6df4f90_MG_R_CC_ANON.tif
File MC done
File noMC done
Number of backgrounds ignored:  1297
Number of positives:  0
Numb